In [1]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
# import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from get_mapping import get_mapping

In [2]:
file_names = ["GDS4987", "GDS4399", "GDS4133", "GDS4132", "GDS3841", "GDS3104", "GDS2084", "GDS1051", "GDS1050"]
file_names.sort()
print("Current Location:", os.getcwd())
os.chdir("../datasets/")
print("Changed Directory To:", os.getcwd())

Current Location: /home/sowmya/Desktop/cs6024/project/codes
Changed Directory To: /home/sowmya/Desktop/cs6024/project/datasets


In [3]:
# Assign mapping between samples and sample type
PCOS_mapping = {'GDS1050': ['GSM27536', 'GSM27537', 'GSM27538', 'GSM27540', 'GSM27541'], 'GDS1051': ['GSM29645', 'GSM29646', 'GSM29647', 'GSM29648', 'GSM29649'], 'GDS2084': ['GSM114834', 'GSM114842', 'GSM114843', 'GSM114847', 'GSM114848', 'GSM114850', 'GSM114852', 'GSM114853'], 'GDS3104': ['GSM156186', 'GSM156187', 'GSM156510', 'GSM156511', 'GSM156512', 'GSM156749', 'GSM156750', 'GSM156751', 'GSM156752', 'GSM156753', 'GSM156763', 'GSM156946', 'GSM156948', 'GSM156949', 'GSM156950', 'GSM156951'], 'GDS3841': ['GSM277460', 'GSM277459', 'GSM277458', 'GSM277457', 'GSM277456', 'GSM277455', 'GSM277454', 'GSM277453', 'GSM277452', 'GSM277451', 'GSM277450', 'GSM277449'], 'GDS4132': ['GSM201542', 'GSM201543', 'GSM201544', 'GSM201545', 'GSM201829', 'GSM201830', 'GSM201831', 'GSM201832', 'GSM201833', 'GSM201834'], 'GDS4133': ['GSM201863', 'GSM201864', 'GSM201865', 'GSM201866', 'GSM201867', 'GSM201868', 'GSM201869', 'GSM201870', 'GSM201871', 'GSM201872'], 'GDS4399': ['GSM850530', 'GSM850531', 'GSM850532', 'GSM850533', 'GSM850534', 'GSM850535', 'GSM850536'], 'GDS4987': ['GSM1174423', 'GSM1174424', 'GSM1174425', 'GSM1174426', 'GSM1174427', 'GSM1174428', 'GSM1174429', 'GSM1174430', 'GSM1174431', 'GSM1174432', 'GSM1174433', 'GSM1174434', 'GSM1174435', 'GSM1174436']}
control_mapping = {'GDS1050': ['GSM27531', 'GSM27532', 'GSM27533', 'GSM27534', 'GSM27543', 'GSM27546', 'GSM27548', 'GSM27549'], 'GDS1051': ['GSM29537', 'GSM29638', 'GSM29643', 'GSM29644', 'GSM29650', 'GSM29651', 'GSM29652', 'GSM29653'], 'GDS2084': ['GSM114841', 'GSM114844', 'GSM114845', 'GSM114849', 'GSM114851', 'GSM114854', 'GSM114855'], 'GDS3104': ['GSM155631', 'GSM155643', 'GSM155644', 'GSM155729', 'GSM156170', 'GSM156171', 'GSM156176', 'GSM156177', 'GSM156178', 'GSM156179', 'GSM156180', 'GSM156181', 'GSM156184'], 'GDS3841': ['GSM277448', 'GSM277447', 'GSM277446', 'GSM277445', 'GSM277444', 'GSM277443', 'GSM277442', 'GSM277441', 'GSM277440', 'GSM277439', 'GSM277438'], 'GDS4132': [''], 'GDS4133': ['GSM201849', 'GSM201850', 'GSM201851', 'GSM201852', 'GSM201853', 'GSM201854', 'GSM201855', 'GSM201856', 'GSM201857', 'GSM201858', 'GSM201859', 'GSM201861', 'GSM201862'], 'GDS4399': ['GSM850527', 'GSM850528', 'GSM850529'], 'GDS4987': ['GSM1174423', 'GSM1174424', 'GSM1174425', 'GSM1174426', 'GSM1174427', 'GSM1174428', 'GSM1174429', 'GSM1174430', 'GSM1174431', 'GSM1174432', 'GSM1174433', 'GSM1174434', 'GSM1174435', 'GSM1174436']}

# Get mapping between the IDs provided and the Entrez Gene IDs
mapping, exclude_mapping = get_mapping(flag=True)
unique_geneids = list(set(mapping.values()))

def get_float_or_str(j):
    if type(j) == float:
        return int(j)
    elif type(j) == str and len(j.split("_")) == 1:
        return (int(j))
    else:
        return j
unique_geneids = [get_float_or_str(i) for i in unique_geneids]
print(len(unique_geneids))

Number of IDs excluded: 20665
Number of genes mapped: 67313
Number of unique gene IDs: 25180
25180


In [4]:
df_list = []
for i in file_names:
    print(i)
    fin = open(i + "_full.txt")
    complete_data = fin.read().splitlines()
    fin.close()
    
    # Find position from which the dataset tabular form begins
    pos = complete_data.index("!dataset_table_begin")
    data = [i.split("\t") for i in complete_data[pos+1:-1]]
    df = pd.DataFrame(data[1:], columns=data[0])
    
    # Remove the last 19 columns
    new_columns = list(df.columns[:-19])
    # Remove the column at position 1 - IDENTIFIER
    new_columns.pop(1)
    PCOS = []
    
    # Additionally ignore the last 1 column
    # Get the mapping - PCOS, Normal, Other == [1, 0, -1]
    for j in new_columns[1:-1]:
        if j in PCOS_mapping[i]: 
            PCOS.append(1)
        elif j in control_mapping[i]: 
            PCOS.append(0)
        else:
            PCOS.append(-1)
    
    # Set df to be only equal to the required columns
    df = df[new_columns[:-1]]

    # Replace the ID_REF using the geneid_mapping
    # df.replace({"ID_REF":mapping}, inplace=True) # Has a large overhead. Using map instead
    df["ID_REF"] = df["ID_REF"].map(lambda x: mapping.get(x, np.nan))
    display(df)
    
    # Remove all columns that have a mapping to null :)
    # Basically remove columns that don't map to a valid Entrez ID
    print("Removing all columns that have no Entrez mapping ...")
    df.drop(index=np.where(pd.isnull(df["ID_REF"]))[0], inplace=True)
    display(df)
    
    print("Unique IDs:", len(set(df["ID_REF"])), "Actual Size:", len(df["ID_REF"]))
    # a = list(np.where(df["ID_REF"].duplicated())[0])
    # values = df["ID_REF"].reset_index().loc[a]["ID_REF"]
    # print("Duplicates:", values)
    
    for col in new_columns[1:-1]:
        try:
            df[col] = df[col].astype(float)
        except:
            df[col] = df[col].str.replace("null", str(np.nan)).astype(float)
    
    print("Eliminating all duplicates, replacing them with averages ...")
    df = df.groupby("ID_REF").mean().reset_index()
    display(df)
    
    print("Checking for spurious gene IDs (if any) ...")
    for name in df["ID_REF"]:
        if name not in unique_geneids:
            print(name)
    
    print("Small Additional Crosscheck ...")
    print(np.where(df["ID_REF"].isin([6071])))
    
    display(df)
    
    for col in new_columns[1:-1]:
        try:
            df[col] = df[col].astype(float)
        except:
            df[col] = df[col].str.replace("null", str(np.nan)).astype(float)
    
    print("Eliminating all duplicates, replacing them with averages ...")
    df = df.groupby("ID_REF").mean().reset_index()
            
    display(df)
    
    # Transpose and add PCOS mapping column
    df = df[new_columns[:-1]].T 
    df.columns = df.iloc[0]
    df = df[1:]
    
    print("Scanning for additional columns to add ...")
    time.sleep(2)
    for j in tqdm(unique_geneids):
        if (type(j) == str or type(j) == int) and (j not in df.columns):
            df[j] = np.nan
    
    df = df[unique_geneids]
    
    df["PCOS"] = PCOS
    df.index.rename('sample_id', inplace=True)
    
    display(df)
    print("="*100)
    
    df.to_csv(i + ".csv")
    df_list.append(df)

GDS1050


,ID_REF,GSM27536,GSM27537,GSM27538,GSM27540,GSM27541,GSM27531,GSM27532,GSM27533,GSM27534,GSM27543,GSM27546,GSM27548,GSM27549
0,780_100616237,540.3,801,701.4,540.9,570,676.6,769.2,604.3,584.2,699,830.5,600.9,612
1,5982,98.9,48.2,90.9,53.8,57.5,39,45.2,50.1,51,81.8,80.2,97.7,59
2,3310,75.8,39.5,36.2,21.6,56.2,41.3,32.6,77.7,57.5,74.6,11.8,49.4,46.1
3,7849,561.3,433.6,395.6,414.6,606.6,585.3,527,501.3,506.5,743,696.3,736.3,757
4,2978,38.2,26.3,22,17.8,24.5,28.6,24.9,14,14.4,43.5,62,67.6,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22278,NaN,5.7,4.3,3.5,1.7,2.8,6.9,2.4,4.1,6.9,6.9,6.1,5,4.8
22279,NaN,17.4,7.5,12,17.9,20.7,15.4,12.1,13.3,3,9,22,17.9,27
22280,NaN,2,0.9,8.3,0.6,4.7,6.1,1.7,1.1,0.8,6.1,11.4,12,1.3
22281,NaN,8.2,5.4,14.6,2.1,26.9,3.9,3.6,3.7,4.5,8.7,8.1,4.2,4.9


Removing all columns that have no Entrez mapping ...


,ID_REF,GSM27536,GSM27537,GSM27538,GSM27540,GSM27541,GSM27531,GSM27532,GSM27533,GSM27534,GSM27543,GSM27546,GSM27548,GSM27549
0,780_100616237,540.3,801,701.4,540.9,570,676.6,769.2,604.3,584.2,699,830.5,600.9,612
1,5982,98.9,48.2,90.9,53.8,57.5,39,45.2,50.1,51,81.8,80.2,97.7,59
2,3310,75.8,39.5,36.2,21.6,56.2,41.3,32.6,77.7,57.5,74.6,11.8,49.4,46.1
3,7849,561.3,433.6,395.6,414.6,606.6,585.3,527,501.3,506.5,743,696.3,736.3,757
4,2978,38.2,26.3,22,17.8,24.5,28.6,24.9,14,14.4,43.5,62,67.6,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22232,2597,9624.9,11214.3,8624.7,8831.7,7978.6,7441.8,8402.3,9742.5,9285.3,9197.3,9816.9,7740.6,7795.3
22233,6772,242.8,362.1,447.9,2115.8,320.4,228.8,268,199.8,285,271.5,243.6,288.2,278.7
22234,6772,11.3,15.6,20.5,112.6,40.2,34.3,33.1,42.5,39.9,74.3,54.3,67.6,58.3
22235,6772,13.2,44.1,41.1,288.4,139.1,96.3,106.7,92.2,140.3,130.8,79.3,140.5,120.6


Unique IDs: 13299 Actual Size: 21156
Eliminating all duplicates, replacing them with averages ...


...

Eliminating all duplicates, replacing them with averages ...


,ID_REF,GSM1174425,GSM1174429,GSM1174436,GSM1174427,GSM1174430,GSM1174432,GSM1174435,GSM1174424,GSM1174428,...,GSM1174416,GSM1174419,GSM1174408,GSM1174413,GSM1174417,GSM1174420,GSM1174410,GSM1174411,GSM1174415,GSM1174422
0,2,2.818150,2.887690,2.980650,-3.525380,-3.521330,-3.566470,-3.429890,1.554540,2.080120,...,-1.869070,-2.228090,1.845970,2.282530,2.043430,1.916010,-1.745030,-1.050880,-0.415323,-0.368802
1,12,0.000000,0.014427,-0.116639,2.072260,1.418320,3.007250,0.717401,-0.133122,0.086389,...,3.846320,3.223370,-0.480368,-0.354922,-0.213374,0.183758,-0.405042,-0.405701,-0.063835,-0.410328
2,16,0.208924,0.735067,-0.115965,-0.761099,-0.280028,-1.158230,-0.522668,-0.435685,0.411367,...,-0.202639,-0.188616,0.430524,0.629131,0.447808,0.475255,0.334290,-0.439426,0.258777,-0.202018
3,18,-0.180470,0.129314,0.064477,0.009513,0.048908,-0.039592,0.004148,0.125017,0.300837,...,0.143869,-0.043517,0.051072,-0.024912,-0.023784,-0.020250,-0.023128,-0.024912,0.129123,-0.197569
4,21,0.318855,0.174927,0.222412,-0.049681,0.050637,-0.074867,0.204762,0.009822,-0.100112,...,0.000000,-0.194595,-0.008076,-0.180850,0.174923,-0.022442,0.111007,0.031285,-0.275916,-0.092824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7927,9556,-0.618012,-0.598186,-0.453303,0.346010,0.159875,0.367265,0.069210,-0.202255,-0.249895,...,-0.006843,0.308805,0.006623,-0.119037,0.169888,0.095421,0.507918,0.514419,0.215421,0.604333
7928,9659_100996724,0.023486,-0.112501,-0.270994,0.023322,0.000000,0.093695,0.357036,-0.047477,0.358214,...,-0.261129,0.141325,-0.030933,0.031506,-0.014572,-0.222528,0.090470,0.092452,0.038302,-0.171990
7929,9698,-0.079279,-0.144554,-0.251414,-0.425434,-0.298251,-0.884743,0.028409,0.066935,0.393334,...,-0.015659,0.200603,-0.030984,0.152534,0.132085,0.182817,-0.326638,-0.909085,-0.344563,-0.424698
7930,9742,-0.035834,-0.160748,0.003976,-0.243402,0.014436,-0.608540,0.057287,-0.110240,0.154281,...,0.195221,0.276109,0.074017,0.000000,0.083637,-0.069797,0.177304,-0.235866,-0.273764,-0.337288


Scanning for additional columns to add ...


100%|██████████| 25180/25180 [00:27<00:00, 917.68it/s] 


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,393046,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM1174425,NaN,2.81815,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174429,NaN,2.88769,NaN,NaN,NaN,NaN,0.0144272,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174436,NaN,2.98065,NaN,NaN,NaN,NaN,-0.116639,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174427,NaN,-3.52538,NaN,NaN,NaN,NaN,2.07226,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174430,NaN,-3.52133,NaN,NaN,NaN,NaN,1.41832,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174432,NaN,-3.56647,NaN,NaN,NaN,NaN,3.00725,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174435,NaN,-3.42989,NaN,NaN,NaN,NaN,0.717401,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174424,NaN,1.55454,NaN,NaN,NaN,NaN,-0.133122,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174428,NaN,2.08012,NaN,NaN,NaN,NaN,0.0863886,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [5]:
df_normalised_list = []
for df,i in zip(df_list, file_names):
    df_new = df[unique_geneids]
    df_new = (df_new - df_new.min())/(df_new.max()-df_new.min())
    df_new["PCOS"] = df["PCOS"]
    display(df_new)
    
    df_new.to_csv(i + "_normalized.csv")
    df_normalised_list.append(df_new)

ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM27536,NaN,0.356383,NaN,NaN,0,0.384615,0.305418,0.0209581,1,0.607143,...,0.309761,NaN,NaN,NaN,1,NaN,0.41055,0.41055,NaN,1
GSM27537,NaN,0.597518,NaN,NaN,0.969251,0.48951,0.919428,0.0419162,0.0173238,0.392857,...,0.447211,NaN,NaN,NaN,0.126031,NaN,0.0389908,0.0389908,NaN,1
GSM27538,NaN,0.365248,NaN,NaN,0.260695,0,1,0,0,0.178571,...,0.699203,NaN,NaN,NaN,0.62662,NaN,0.963303,0.963303,NaN,1
GSM27540,NaN,0.150709,NaN,NaN,1,0.241259,0,0.0568862,0.100358,0.142857,...,0.445219,NaN,NaN,NaN,0.246172,NaN,1,1,NaN,1
GSM27541,NaN,0,NaN,NaN,0.628342,0.234266,0.115201,0.113772,0.146953,0.357143,...,0.61255,NaN,NaN,NaN,0.23086,NaN,0.497706,0.497706,NaN,1
GSM27531,NaN,0.602837,NaN,NaN,0.455882,0.143357,0.62304,0.0329341,0.666069,0.142857,...,0.462151,NaN,NaN,NaN,0.47821,NaN,0.0688073,0.0688073,NaN,0
GSM27532,NaN,0.223404,NaN,NaN,0.22861,0.374126,0.278428,0.0598802,0.233572,0.410714,...,0.511952,NaN,NaN,NaN,0.240283,NaN,0.518349,0.518349,NaN,0
GSM27533,NaN,0.624113,NaN,NaN,0.165775,0.0804196,0.636833,0.110778,0.673238,0,...,0.378486,NaN,NaN,NaN,0.213192,NaN,0.192661,0.192661,NaN,0
GSM27534,NaN,0.445035,NaN,NaN,0.229947,0.241259,0.518158,0.00898204,0.502389,0.339286,...,0.437251,NaN,NaN,NaN,0,NaN,0.133028,0.133028,NaN,0


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM29645,0.359551,NaN,0.694396,0.0182983,NaN,NaN,NaN,NaN,NaN,NaN,...,0.542205,1,0,0.302326,NaN,NaN,NaN,NaN,0.0896899,1
GSM29646,0.579243,NaN,0.421144,0.912168,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0.788192,1,0.17759,NaN,NaN,NaN,NaN,0.927913,1
GSM29647,0.418096,NaN,0.736568,0.209515,NaN,NaN,NaN,NaN,NaN,NaN,...,0.612196,0.0819188,0.0672043,0.133192,NaN,NaN,NaN,NaN,0.2171,1
GSM29648,0.176227,NaN,0.320624,0.531565,NaN,NaN,NaN,NaN,NaN,NaN,...,0.428958,0.826568,0.865591,0.139535,NaN,NaN,NaN,NaN,0.590947,1
GSM29649,0,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.541904,0.831734,0.142473,0.0295983,NaN,NaN,NaN,NaN,0.0620285,1
GSM29537,0.466884,NaN,0.186597,0.589204,NaN,NaN,NaN,NaN,NaN,NaN,...,0.505257,0.548339,0.104839,0.192389,NaN,NaN,NaN,NaN,0,0
GSM29638,0.235068,NaN,0.493356,0.0375114,NaN,NaN,NaN,NaN,NaN,NaN,...,0.248123,0.115867,0.0349462,0.477801,NaN,NaN,NaN,NaN,0.380553,0
GSM29643,0.338853,NaN,0.556904,0.348582,NaN,NaN,NaN,NaN,NaN,NaN,...,0.3845,0.648708,0.107527,1,NaN,NaN,NaN,NaN,0.323554,0
GSM29644,0.735068,NaN,0.126516,0.130833,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0.0833333,0,NaN,NaN,NaN,NaN,0.29254,0


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM114841,NaN,1,NaN,NaN,1,0.367089,0.143934,0.563865,1,0.169811,...,0.788337,NaN,NaN,NaN,0.278626,NaN,0.0487805,0.0487805,NaN,0
GSM114844,NaN,0.208973,NaN,NaN,0.414013,0.227848,0.216277,0.146288,0.100128,0.245283,...,1,NaN,NaN,NaN,0.100509,NaN,0.207317,0.207317,NaN,0
GSM114845,NaN,0.125693,NaN,NaN,0.61465,0.822785,0.342879,0.00927948,0.708601,1,...,0.354212,NaN,NaN,NaN,0.00508906,NaN,0.695122,0.695122,NaN,0
GSM114849,NaN,0.423746,NaN,NaN,0.302548,0.291139,0.169555,0.510917,0.0218228,0.0660377,...,0.585313,NaN,NaN,NaN,1,NaN,0.817073,0.817073,NaN,0
GSM114851,NaN,0.226505,NaN,NaN,0.270701,0,0.320271,0.125546,0.603338,0.132075,...,0.652268,NaN,NaN,NaN,0.388041,NaN,0.097561,0.097561,NaN,0
GSM114854,NaN,0.386747,NaN,NaN,0.0923567,0.658228,0.157498,0.477074,0.501926,0.0471698,...,0.732181,NaN,NaN,NaN,0.259542,NaN,0.414634,0.414634,NaN,0
GSM114855,NaN,0.438701,NaN,NaN,0.525478,0.164557,1,0.00218341,0.0410783,0.0943396,...,0.490281,NaN,NaN,NaN,0.28117,NaN,0.0853659,0.0853659,NaN,0
GSM114834,NaN,0.0286193,NaN,NaN,0.684713,0.0886076,0,0.762828,0.165597,0,...,0,NaN,NaN,NaN,0.689567,NaN,1,1,NaN,1
GSM114842,NaN,0.805466,NaN,NaN,0.210191,0.607595,0.10324,0.186681,0.296534,0.179245,...,0.766739,NaN,NaN,NaN,0,NaN,0.280488,0.280488,NaN,1


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM155631,0.653276,0.810938,0.168546,0.350665,0.272959,0.341838,0.730945,0.744826,0.736345,0.711208,...,0.682032,0.446259,0.375688,0.508493,0.455459,NaN,0.734211,0.734211,0.30879,0
GSM155643,0.353426,0.61021,0.0900933,0.405875,0.126684,0.6732,0.149307,0.669677,0.297598,0.381989,...,0.565364,0.109711,0.0720985,0.201682,0.18087,NaN,0.781312,0.781312,0.118327,0
GSM155644,0.846963,0.564647,0.0302528,0.366577,0.128925,0.385669,0.121022,0.678778,0.768503,0.590598,...,0.54798,0,0.398746,0.303901,0.290844,NaN,0.818469,0.818469,0.111728,0
GSM155729,0.351596,0.558253,1,0.278087,1,1,0.235161,0.937086,0,0.420498,...,0,1,0.0758657,0.900452,0.043677,NaN,1,1,1,0
GSM156170,0.472015,0.245801,0.46289,0.443142,0.668163,0.479191,0.296338,0.547432,0.120795,0.513063,...,0.571991,0.537614,0.503406,0.180798,0.48695,NaN,0.468456,0.468456,0.376739,0
GSM156171,0.581927,0.738539,0.446367,0.998472,0.592186,0.656963,0.401715,0.688473,0.324304,0.785003,...,0.739396,0.799523,0.231473,0.310821,0.018535,NaN,0.542001,0.542001,0.416388,0
GSM156176,0.756673,0.378694,0.77788,0.133928,0.615433,0.403687,0.295478,0.604801,0.63366,0.61417,...,0.254742,0.858968,1,0.62425,0.283495,NaN,0.633578,0.633578,0.311295,0
GSM156177,0.395325,0.374407,0.0473389,0.327033,0.262276,0.19006,0,0.654151,0.0863477,0.510837,...,0.818101,0.0122186,0.126702,0.272363,0.169886,NaN,0.59029,0.59029,0.078271,0
GSM156178,0.0803662,0.616452,0.442229,0.514069,0.838356,0.936374,0.21448,1,0.00450444,0.0341097,...,0.411011,0.742203,0.205075,0.759293,0.359592,NaN,0.92339,0.92339,0.841863,0


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM277438,1,1,1,0.322851,0,0.532115,0.479324,0.667352,0.593257,0.248661,...,0.827365,0,0.693762,0.718163,0.163969,NaN,0.29779,0.29779,0.33255,0
GSM277439,0.653275,0.16758,0.165049,0.327351,0.34417,0.342526,0.449352,0.539547,0.540561,0.601389,...,0.740126,0.804617,0.266477,0,0.342748,NaN,0.0407697,0.0407697,0.288548,0
GSM277440,0.523412,0.294368,0.457359,0,0.244294,0.252491,0.420944,0.770855,0.780176,0.175465,...,0.817903,0.959305,0.514504,0.093775,0.210836,NaN,0.213538,0.213538,0.0477384,0
GSM277441,0.225761,0.432814,0.240948,0.362684,0.802241,0.153889,0,0.00163229,0.634459,0.237184,...,0.881571,0.646122,0.460852,0.0105169,0.539008,NaN,0.350066,0.350066,0.267202,0
GSM277442,0.273412,0.139948,0,0.308425,0.810793,0,0.728834,0.106868,0.555612,0.653574,...,0.724989,0.536159,0.271925,0.0067937,0.387518,NaN,0.459792,0.459792,0,0
GSM277443,0,0.105716,0.0513409,0.160863,0.773261,0.230467,0.270302,0,0.582594,0.422143,...,0.651493,0.770906,0.368172,0.144814,0.308082,NaN,0.422993,0.422993,0.62529,0
GSM277444,0.260925,0.298196,0.195198,0.219102,0.824616,0.547188,0.588135,0.410149,0.690468,0.524464,...,0.545892,0.689307,0.403724,0.467909,0.280745,NaN,0.741594,0.741594,0.335011,0
GSM277445,0.584686,0.60515,0.577372,0.498701,0.129667,0.477616,1,0.769483,0.42871,0.513,...,0.160656,0.151039,0.672704,0.556028,0.130192,NaN,0.781478,0.781478,0.992892,0
GSM277446,0.61808,0.368432,0.372631,0.673303,0.5445,0.434083,0.407946,0.425484,0.406537,0.422143,...,0.621144,0.857435,0.520179,0.141905,0.286259,NaN,0.408262,0.408262,0.581022,0


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM201542,0.826925,0.151088,0.55927,0.0975916,0.234521,0.427557,0.338108,0.342172,0.621765,0.778657,...,0.636389,0.0384321,0.553655,0.502746,0.210966,NaN,0.565971,0.565971,0.14677,1
GSM201543,0.750406,0.0369716,0.728029,0,0.0959198,0.537423,0,0.294047,0.683663,0.654951,...,0.49629,0,0.483539,0,0.52838,NaN,0.83897,0.83897,0.125473,1
GSM201544,0.84279,0,0.771164,0.221675,0.311388,1,0.310492,0.0706268,1,0.786379,...,0.634281,0.833532,0.533717,0.449558,0.791723,NaN,0.277485,0.277485,0.308905,1
GSM201545,0.634147,0.709944,0.928383,0.132638,0.551188,0.842405,0.363727,0.510438,0.633251,0.671635,...,0.431754,0.66607,0.735387,1,0.350432,NaN,0.425291,0.425291,0.524615,1
GSM201829,0.542106,0.293777,0.601002,0.597466,0.664567,0.502625,0.202354,0.325754,0.681098,0.411931,...,0.377039,0.529531,0.706835,0.598627,0.489529,NaN,0.646764,0.646764,0.292021,1
GSM201830,0.938848,0.427349,0.879643,0.411562,0.11646,0.566342,0.44412,0.509573,0.832381,1,...,1,0.0865296,0.892659,0.0915167,0.740147,NaN,0.687634,0.687634,0.329061,1
GSM201831,1,0.152763,0.610366,0.0249667,0.188889,0.822493,0.591476,0.689195,0.897457,0.45614,...,0.518426,0.190708,0.248985,0.444879,0.770564,NaN,0.828719,0.828719,0.434739,1
GSM201832,0.845634,0.0338439,0,0.802544,0,0.611758,0.44465,0,0.39104,0.741624,...,0.811761,0.269254,0.566086,0.205892,0.692655,NaN,0,0,0,1
GSM201833,0.447655,0.327205,0.351811,0.304206,0.291637,0.449686,1,0.209782,0.46999,0.388642,...,0.384436,0.600064,0.190032,0.518981,0.731624,NaN,0.495732,0.495732,0.332932,1


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM201849,0.458835,0.378227,0.108546,0.464012,0.0653359,0.848337,0.795135,0.724952,0.480945,0.416413,...,0.75652,0.186684,0,0.00969933,0.165131,NaN,0.853385,0.853385,0.139814,0
GSM201850,0.62777,0.64632,0.181504,0.310362,0.237945,0.215371,1,0.752945,0.782331,0.603324,...,0.849769,0.471523,0.217251,0.387122,0.457359,NaN,0.774817,0.774817,0.311004,0
GSM201851,0.912408,0.333403,0.0941796,0.462528,0.114931,0.457095,0.614251,0.875939,0.884558,0.634409,...,0.82345,0.109827,0.389836,0.16512,0.281907,NaN,0.96701,0.96701,0.151605,0
GSM201852,0.246348,0.518489,1,0.142679,1,1,0.918769,0.915012,0.148423,0.262967,...,0,0.992334,0.541589,1,0.0262282,NaN,1,1,1,0
GSM201853,0.409938,0.223511,0.445634,0.380612,0.725924,0.373072,0.354124,0.0732485,0.193556,0.400482,...,0.60853,0.483193,0.387517,0.177082,0.487667,NaN,0.378221,0.378221,0.318152,0
GSM201854,0.515519,0.586068,0.384389,1,0.762994,0.691629,0.753917,0.490812,0.337458,0.711986,...,0.806375,0.794161,0.0155579,0.14624,0,NaN,0.495144,0.495144,0.374284,0
GSM201855,0.793946,0.267205,0.685058,0.161975,0.81552,0.38149,0.815104,0.566529,0.723982,0.632113,...,0.482999,1,1,0.677681,0.302457,NaN,0.713395,0.713395,0.337897,0
GSM201856,0.420114,0.150908,0.022184,0.31091,0.176371,0,0.167718,0.486073,0.167193,0.478517,...,0.875842,0,0.0320483,0,0.120804,NaN,0.549414,0.549414,0.0531618,0
GSM201857,0.0471771,0.578021,0.451017,0.412948,0.807001,0.916552,0.732974,0.97097,0.104623,0.00847419,...,0.401099,0.678878,0.527989,0.745275,0.358612,NaN,0.864612,0.864612,0.80779,0


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM850527,0.434441,0,1,1,0.695249,0.294968,0.0127694,1,0,1,...,0.228134,0.173647,0.377126,0.193806,0,NaN,1,1,0.0596846,0
GSM850528,0.480817,0.104285,0.0020124,0.284588,0.785497,0.0770181,0.495981,0.534207,0.433449,0.000878662,...,0.416488,1,0.277298,0.102602,0.104889,NaN,0.0177607,0.0177607,0.40066,0
GSM850529,0.589708,0.120093,0,0.284712,0.722574,0.0255069,0.502639,0.293625,0.520687,0.32476,...,0.641144,0.922396,0.240641,0.170715,0.107342,NaN,0,0,1,0
GSM850530,0.534821,0.352074,0.418144,0.317983,0.658039,0.376643,0,0.328219,0.264374,0.0994946,...,0.492877,0.362597,0.514314,0.0707735,0.138151,NaN,0.853761,0.853761,0.931832,1
GSM850531,0.366893,0.506061,0.00748882,0.173828,0.735464,0.00528786,1,0.130208,0.314273,0.112469,...,1,0.421994,0,0.0290011,0.0689843,NaN,0.0263392,0.0263392,0.810939,1
GSM850532,0,0.560055,0.210045,0.147566,0.610551,0.0945177,0.207783,0,0.456851,0.155483,...,0.567993,0.420845,0.561373,0,0.199106,NaN,0.26751,0.26751,0.0472862,1
GSM850533,0.251124,0.47601,0.109241,0.127245,0.492935,0,0.881629,0.396327,0.475477,0,...,0.610309,0.227643,0.446696,0.0616502,0.119653,NaN,0.0183907,0.0183907,0.439585,1
GSM850534,1,1,0.119396,0.0918431,1,0.425885,0.527225,0.541481,0.801295,0.442296,...,0.312233,0,1,0.526725,0.0752053,NaN,0.196361,0.196361,0.394745,1
GSM850535,0.711123,0.757018,0.0973895,0.0593942,0.102724,1,0.385323,0.453658,1,0.143705,...,0.327009,0.0905675,0.886721,0.115905,1,NaN,0.153092,0.153092,0,1


ID_REF,1,2,3,131076,9,10,12,13,14,15,...,130916,51130_100302652,130940,130951,401331_10156_100271927,10720_7364_7365_7366_7367,8345,8345,131034,PCOS
sample_id,,,,,,,,,,,,,,,,,,,,,
GSM1174425,NaN,0.955117,NaN,NaN,NaN,NaN,0.128648,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174429,NaN,0.96552,NaN,NaN,NaN,NaN,0.131916,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174436,NaN,0.979426,NaN,NaN,NaN,NaN,0.102224,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174427,NaN,0.00614692,NaN,NaN,NaN,NaN,0.598101,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174430,NaN,0.00675278,NaN,NaN,NaN,NaN,0.449956,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174432,NaN,0,NaN,NaN,NaN,NaN,0.809916,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174435,NaN,0.0204319,NaN,NaN,NaN,NaN,0.291169,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174424,NaN,0.766085,NaN,NaN,NaN,NaN,0.09849,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
GSM1174428,NaN,0.84471,NaN,NaN,NaN,NaN,0.148218,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
